## Intro

In this post, I replicate  _Salomon Brother's Principles of Principal Components_ 2003 paper on modern data (yields from 2012-2022), which showed how PCA on the yield curve can be used in bond relative value analysis: to structure curve-neutral butterfly trades free of level and slope bias. 

**Corey Hoffstein Tweet picture**

To understand the relevant concepts, I worked through selected chapters of Jha's _Interest Rates Markets_, and later, Gregory Gundersen's _PCA Explained_.

The paper is divided into three parts: explaining how PCA on the yield curve works, using PCs to construct a replicating portfolio, and using PCs as weights for butterfly trades. I focus on the first & third parts.



## PCA to model the yield curve

The first part of the paper explains how PCA quantifies yield curve reshaping. The first 3 PCs correspond to level, slope and curvature respectively. PCA performs eigendecomposition on the covariance matrix:  
$$\mathbf{X}^T\mathbf{X} = \mathbf{Q}\mathbf{\Lambda}\mathbf{Q}^T$$
Where $X$ is the demeaned matrix (column-wise) where the columns are the tenors (e.g 1Y, 3Y .. 30Y) and rows are the daily yields. I used the official [Daily Treasury Par Yield Curve Rates](https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value_month=202312) data. 

The authors used 120 data points along the yield curve (interpolated at 90-day intervals with a cubic spline). We did so as well and performed PCA. Each PC is a vector in 120 dimensional space. We replicate this plot:

**PCA cum. var plot and PC plot across tenor**

To link each PCA to the correct movement: note the sign - the first PC (level) never changes sign. The second (slope) changes sign once, and third (curvature) changes sign twice. Projecting our yields in PC space is simply $\Lambda_{1,2,3} X=P$.

**Projected PCA data across time | PC2 vs Slope**

We can reexpress our yield curve data in PC space. To confirm the idea, we examine the z-scored the 10Y-2Y spread (slope proxy) against PC2.

**Yields over time plot**

Note flattening of the curve from 2014-2019 and a sharp drop in yield across all tenors in 2020 due to Covid and aggressive Fed rate cuts. Now observe PC2 and PC1 in these time periods on the first chart. 


## PCA for Butterfly Trades

### Relative Value Trades

An investor can express a view on relative performance via a multi-legged trade. In a butterfly, the investor takes a view on _curvature_.

**Jha diagram**

If we expect curvature to hump more, the belly (5s) yields go up and wing (2s, 10s) yields go down, thus one sells the belly and buys the wings. However, weights for each leg must be determined.

### PCAs as Weights

The authors use the 3rd PC loadings as weights for the trade - in theory these are uncorrelated to market or slope changes, creating a butterfly based off pure curvature.

The paper states 'the trading signals are obtained from the percentiles of the (PC-weighted) butterfly spread over a rolling window of a specified length (2Y, 3Y, 4Y)'. The authors then iterated over various parameters: tenors, rolling window length, entry/exit levels, holding periods, levels vs changes, etc. More importantly, they plotted some charts to reference: This is an example of a mean-reverting butterfly from the paper: a 4s-10s-27s on yields from 1997 to 1998. The 2nd/3rd plots show exposure to level and slope. 

**SSB 8 charts**

We thus experiment with different rolling window lengths for the 2s-5s-10s trade from 2017-2022.  The code being fairly simple:

**PCA code**

We then create the spread for different rolling window lengths.


**Butterfly for diff rolling windows**

The procedure was as such:

1. Perform rolling PCA on cov. matrix of selected yields: 2s-5s-10s. 
2. Extract PC3 loadings and normalize the weights using the belly. E.g if our loadings are `[-0.63, 0.70, -0.32]`, our weights are `[-0.90, 1, -0.46]`
3. Dot product weights with yields to get butterfly spread
4. Plot out the fly over time, check if stationary.
5. Generate signals/positions and backtest.
 
 Regarding point 5, PCA weights are _risk weights_ or _duration dollar_ weights. They must be converted to _notional weights_ via a DV01 calculation. These are then used to calculate returns. However, calculating DV01/duration requires prices/coupon rates which are only accessible with a Bloomberg terminal. Fortunately, as our spreads are not mean-reverting, any trades would be unprofitable, hence, we do not calculate returns/backtest.



Unfortunately, these were _not_ mean reverting, especially so for a single year:

**2021 spreads**

However, the spread does seem somewhat mean-reverting over a 5Y period for a 1Y rolling window:

**My triple charts**.

The high correlation coefficients are in constrast to the zero to no correlation in the original paper:

**SSB Charts**

We then plotted out the weights over time for this 1Y rolling window to get a feel of what was going on. We try an alternative: what if we z-score the weights (over time) over a rolling window? The resultant weights would look like this.

**Weights vs Rolling weights**

And as seen below, these weights give us a very stationary spread. However, these weights are no longer a butterfly, as the signs are of wings are no longer both -ve with a +ve belly:

**Z-spread**



# Conclusion

In conclusion, this post replicates the butterfly trades aspect of the Principals of Principal Components by Salomon Brothers.

It is clear yield curve dynamics are very different now as compared to 20 years ago. As such, the rolling PCA risk weights failed to give us a stationary spread to trade. Nonetheless, this was a significant milestone for me - my first paper replication. From reading up on relevant material (Jha) to understand concepts, to finally digesting the paper, to deciphering how to code out the research and replicating it, I have learnt a great deal about fixed income basics and PCA.
